# Install dependencies

In [1]:
%pip install openai

In [2]:
%pip install qdrant-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.2/267.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 15.6 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling protobuf-4.25.5:
      Successfully uninstalled protobuf-4.25.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorboard 2.17.0 requires protobuf!=4.24.0,<5.0.0,>=3.19.6, but you have protobuf 5.28.3 which is incompatible.
tensorflow 2.17.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.28.3 which is incompatible.
tensorflow-metadata 1.13.1 requires protobuf<5,>=3.20.3, but you have p

# Code

In [ ]:
from qdrant_client import QdrantClient
from google.colab import userdata

client = QdrantClient(
    url=userdata.get('QDRANT_URL'),
    api_key=userdata.get('QDRANT_API_KEY'),
)

collection_name = "medellin-info-collection"

In [35]:
import json
import glob

# Lee y procesa el archivo JSON
def cargar_json(ruta_archivo):
    with open(ruta_archivo, 'r', encoding='UTF-8') as file:
        data = json.load(file)
    return data

In [36]:
nombre_archivo = 'Diccionario_paisa'
ruta_archivo = f'/content/{nombre_archivo}.json'
json_data = cargar_json(ruta_archivo)

In [37]:
json_data

[{'title': 'Amañar',
  'text': 'Este verbo en el vocabulario paisa significa sentirse muy a gusto en un lugar, tanto como para tener la intención de permanecer allí durante mucho tiempo. Ejemplo: ¡Me amañé mucho en Colombia, me voy a quedar!',
  'category': 'Paisa_dictionary'},
 {'title': 'Bacano',
  'text': 'Probablemente una de las palabras más conocidas de la jerga colombiana es bacano, ésta es usada prácticamente en cualquier región y quiere decir que algo es muy bueno o es una cosa de la que sentimos mucho agrado. Ejemplo: ¡Ir al Estadio en Medellín, es muy bacano!',
  'category': 'Paisa_dictionary'},
 {'title': 'Bonito',
  'text': 'Esta palabra no solo tiene un significado simple de “bello” o “agradable”, bonito también se refiere a algo bien hecho, que actúa bien. Ejemplo: ¡Medellín tan bonito, pero más bonito tratarla con respeto!',
  'category': 'Paisa_dictionary'},
 {'title': 'Cacharro',
  'text': 'Un cacharro hace referencia a una situación que deriva en una experiencia que 

In [38]:
def concatenar_llaves_valor_json(objeto_json):
  concatenated_text = "".join(f"{k}: [{v}] " for k, v in objeto_json.items())
  return concatenated_text

In [39]:
# Combina todas las claves y valores de un json en un solo texto
objeto_json = json_data[0]
concatenated_text = concatenar_llaves_valor_json(objeto_json)
concatenated_text

'title: [Amañar] text: [Este verbo en el vocabulario paisa significa sentirse muy a gusto en un lugar, tanto como para tener la intención de permanecer allí durante mucho tiempo. Ejemplo: ¡Me amañé mucho en Colombia, me voy a quedar!] category: [Paisa_dictionary] '

In [40]:
import openai

openai_client = openai.Client(
    api_key=userdata.get('OPENAI_API_KEY')
)

In [41]:
# Genera embeddings para un texto
embedding_model = "text-embedding-3-small"
def generar_embedding(texto):
  result = openai_client.embeddings.create(input=texto, model=embedding_model)
  return result.data[0].embedding

In [42]:
embedding = generar_embedding(concatenated_text)
embedding

[0.035169005393981934,
 0.013895048759877682,
 -0.04111753776669502,
 0.01682390831410885,
 -0.013316088356077671,
 0.01479187048971653,
 0.024656901136040688,
 -0.03385215252637863,
 -0.05798685550689697,
 -0.007168891839683056,
 -0.02785821072757244,
 -0.017834249883890152,
 0.019991161301732063,
 0.0031644157133996487,
 0.02847122773528099,
 0.035713907331228256,
 -0.008099769242107868,
 -0.005727166775614023,
 0.03140008822083473,
 0.013917753472924232,
 0.016892019659280777,
 0.048087768256664276,
 -0.024566084146499634,
 0.04895053058862686,
 0.026677586138248444,
 -0.014110740274190903,
 -0.031944990158081055,
 0.07442478835582733,
 0.015302717685699463,
 -0.04252520576119423,
 -0.010903753340244293,
 -0.035327933728694916,
 -0.010744823142886162,
 -0.057805221527814865,
 -0.011454333551228046,
 -0.00154389429371804,
 0.016528751701116562,
 -0.05299190431833267,
 -0.02819877676665783,
 -0.07642277330160141,
 -0.031672537326812744,
 -0.02139882929623127,
 0.04849644750356674,
 0.

In [43]:
import uuid

# Procesa cada JSON y le agrega el embedding
def process_and_modify_json_data(json_data):
    for objeto_json in json_data:
        # Combina todas las claves y valores en un solo texto
        texto_concatenado = concatenar_llaves_valor_json(objeto_json)

        # Genera el embedding del texto concatenado
        embedding = generar_embedding(texto_concatenado)

        # Agrega el embedding al objeto JSON
        objeto_json['embedding'] = embedding

        # Agrega un UUID como id al objeto JSON
        objeto_json['id'] = str(uuid.uuid4())

    return json_data  # Retorna los JSON enriquecidos con embeddings

In [44]:
json_data = cargar_json(ruta_archivo)
modified_json_data = process_and_modify_json_data(json_data)
modified_json_data

[{'title': 'Amañar',
  'text': 'Este verbo en el vocabulario paisa significa sentirse muy a gusto en un lugar, tanto como para tener la intención de permanecer allí durante mucho tiempo. Ejemplo: ¡Me amañé mucho en Colombia, me voy a quedar!',
  'category': 'Paisa_dictionary',
  'embedding': [0.035169005393981934,
   0.013895048759877682,
   -0.04111753776669502,
   0.01682390831410885,
   -0.013316088356077671,
   0.01479187048971653,
   0.024656901136040688,
   -0.03385215252637863,
   -0.05798685550689697,
   -0.007168891839683056,
   -0.02785821072757244,
   -0.017834249883890152,
   0.019991161301732063,
   0.0031644157133996487,
   0.02847122773528099,
   0.035713907331228256,
   -0.008099769242107868,
   -0.005727166775614023,
   0.03140008822083473,
   0.013917753472924232,
   0.016892019659280777,
   0.048087768256664276,
   -0.024566084146499634,
   0.04895053058862686,
   0.026677586138248444,
   -0.014110740274190903,
   -0.031944990158081055,
   0.07442478835582733,
   0.0

In [45]:
modified_json_data[0]

{'title': 'Amañar',
 'text': 'Este verbo en el vocabulario paisa significa sentirse muy a gusto en un lugar, tanto como para tener la intención de permanecer allí durante mucho tiempo. Ejemplo: ¡Me amañé mucho en Colombia, me voy a quedar!',
 'category': 'Paisa_dictionary',
 'embedding': [0.035169005393981934,
  0.013895048759877682,
  -0.04111753776669502,
  0.01682390831410885,
  -0.013316088356077671,
  0.01479187048971653,
  0.024656901136040688,
  -0.03385215252637863,
  -0.05798685550689697,
  -0.007168891839683056,
  -0.02785821072757244,
  -0.017834249883890152,
  0.019991161301732063,
  0.0031644157133996487,
  0.02847122773528099,
  0.035713907331228256,
  -0.008099769242107868,
  -0.005727166775614023,
  0.03140008822083473,
  0.013917753472924232,
  0.016892019659280777,
  0.048087768256664276,
  -0.024566084146499634,
  0.04895053058862686,
  0.026677586138248444,
  -0.014110740274190903,
  -0.031944990158081055,
  0.07442478835582733,
  0.015302717685699463,
  -0.04252520

In [46]:
output_directory = '/content/Embeddings'

In [47]:
# Guardar los JSONs enriquecidos localmente
with open(f'{output_directory}/{nombre_archivo}_embedding.json', 'w', encoding='utf-8') as archivo:
    json.dump(modified_json_data, archivo, ensure_ascii=False, indent=2)

In [50]:
# Create qdrant point
from qdrant_client.http.models import PointStruct

def create_point(json_object):
  # Create payload with all dict elements except id and embeddig
  payload = {k: v for k, v in json_object.items() if k not in ['id', 'embedding']}
  # Create point
  point = PointStruct(
      id=json_object.get("id"),
      vector=json_object.get("embedding"),
      payload=payload
  )
  return point

In [51]:
example_point = create_point(modified_json_data[0])
example_point

PointStruct(id='949c6222-be31-4dbf-9ab9-18bbf89557b2', vector=[0.035169005393981934, 0.013895048759877682, -0.04111753776669502, 0.01682390831410885, -0.013316088356077671, 0.01479187048971653, 0.024656901136040688, -0.03385215252637863, -0.05798685550689697, -0.007168891839683056, -0.02785821072757244, -0.017834249883890152, 0.019991161301732063, 0.0031644157133996487, 0.02847122773528099, 0.035713907331228256, -0.008099769242107868, -0.005727166775614023, 0.03140008822083473, 0.013917753472924232, 0.016892019659280777, 0.048087768256664276, -0.024566084146499634, 0.04895053058862686, 0.026677586138248444, -0.014110740274190903, -0.031944990158081055, 0.07442478835582733, 0.015302717685699463, -0.04252520576119423, -0.010903753340244293, -0.035327933728694916, -0.010744823142886162, -0.057805221527814865, -0.011454333551228046, -0.00154389429371804, 0.016528751701116562, -0.05299190431833267, -0.02819877676665783, -0.07642277330160141, -0.031672537326812744, -0.02139882929623127, 0.04

In [59]:
example_point.payload["title"]

'Amañar'

In [64]:
"""client.upsert(
    collection_name=collection_name,
    points=[example_point]
)
"""

'client.upsert(\n    collection_name=collection_name,\n    points=[example_point]\n)\n'

In [67]:
# Subir cada json a qdrant
from qdrant_client.http.models import PointStruct
def create_points_and_upload(json_data):
    puntos_qdrant = []

    for objeto_json in json_data:
        # Crea un punto para subir a Qdrant
        punto = create_point(objeto_json)
        puntos_qdrant.append(punto)

    # Subir todos los puntos a la colección de Qdrant
    client.upsert(
        collection_name=collection_name,
        points=puntos_qdrant
    )

In [68]:
create_points_and_upload(modified_json_data)

# Pruebas

In [70]:
example_text = "Puedes explicarme que significa la palabra parcero?"
example_embedding = openai_client.embeddings.create(
    input=example_text,
    model=embedding_model,
).data[0].embedding

In [73]:
result = client.search(
    collection_name=collection_name,
    query_vector=example_embedding,
    limit=5,
)
result

[ScoredPoint(id='e9528118-176d-4585-895f-bc42c474a631', version=6, score=0.70152515, payload={'title': 'Parcero', 'text': '¡Esta palabra nos encanta! Ser un parcero o parcera, es ser un amigo, fiel, del alma, que te cuida y que siempre está ahí. Ejemplo: “Vos sos mi parcero, siempre me cuidás”', 'category': 'Paisa_dictionary'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id='f1c56b47-6395-4264-895d-df511d9e6881', version=6, score=0.48496088, payload={'title': 'Parce', 'text': 'Otra de las palabras más famosas de Medellín es parce, la cual quiere decir amigo, compañero o camarada, y es usada para referirse con confianza a otra persona. Ejemplo: Parce, pasame las llaves.', 'category': 'Paisa_dictionary'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id='c5fcd923-2278-4a9a-9f7a-861d2b4c4b60', version=6, score=0.48068342, payload={'title': 'Parche', 'text': 'En Medellín un parche puede ser varias cosas, pero todas tienen que ver con entretenimiento, así por

# Funciones ordenadas

In [74]:
from qdrant_client import QdrantClient
from google.colab import userdata
import openai

openai_client = openai.Client(
    api_key=userdata.get('OPENAI_API_KEY')
)
client = QdrantClient(
    url="https://31f8fa0f-60fa-4eaa-ae63-81a27026b200.us-east4-0.gcp.cloud.qdrant.io:6333",
    api_key=userdata.get('QDRANT_API_KEY'),
)

In [75]:
import json
import glob

# Lee y procesa el archivo JSON
def cargar_json(ruta_archivo):
    with open(ruta_archivo, 'r', encoding='UTF-8') as file:
        data = json.load(file)
    return data

In [76]:
def concatenar_llaves_valor_json(objeto_json):
  concatenated_text = "".join(f"{k}: [{v}] " for k, v in objeto_json.items())
  return concatenated_text

In [77]:
# Genera embeddings para un texto
def generar_embedding(texto):
  result = openai_client.embeddings.create(input=texto, model=embedding_model)
  return result.data[0].embedding

In [78]:
import uuid

# Procesa cada JSON y le agrega el embedding
def process_and_modify_json_data(json_data):
    for objeto_json in json_data:
        # Combina todas las claves y valores en un solo texto
        texto_concatenado = concatenar_llaves_valor_json(objeto_json)

        # Genera el embedding del texto concatenado
        embedding = generar_embedding(texto_concatenado)

        # Agrega el embedding al objeto JSON
        objeto_json['embedding'] = embedding

        # Agrega un UUID como id al objeto JSON
        objeto_json['id'] = str(uuid.uuid4())

    return json_data  # Retorna los JSON enriquecidos con embeddings

In [79]:
# Create qdrant point
from qdrant_client.http.models import PointStruct

def create_point(json_object):
  # Create payload with all dict elements except id and embeddig
  payload = {k: v for k, v in json_object.items() if k not in ['id', 'embedding']}
  # Create point
  point = PointStruct(
      id=json_object.get("id"),
      vector=json_object.get("embedding"),
      payload=payload
  )
  return point

In [80]:
# Subir cada json a qdrant
from qdrant_client.http.models import PointStruct
def create_points_and_upload(json_data):
    puntos_qdrant = []

    for objeto_json in json_data:
        # Crea un punto para subir a Qdrant
        punto = create_point(objeto_json)
        puntos_qdrant.append(punto)

    # Subir todos los puntos a la colección de Qdrant
    client.upsert(
        collection_name=collection_name,
        points=puntos_qdrant
    )

In [81]:
embedding_model = "text-embedding-3-small"
collection_name = "medellin-info-collection"
output_directory = '/content/Embeddings'

In [94]:
nombre_archivo = 'Turismo_responsable'
ruta_archivo = f'/content/{nombre_archivo}.json'

In [95]:
# Main
json_data = cargar_json(ruta_archivo)
modified_json_data = process_and_modify_json_data(json_data)
create_points_and_upload(modified_json_data)

In [96]:
# Guardar los JSONs enriquecidos localmente
with open(f'{output_directory}/{nombre_archivo}_embedding.json', 'w', encoding='utf-8') as archivo:
    json.dump(modified_json_data, archivo, ensure_ascii=False, indent=2)